Import Dependencies

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import models

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [25]:
# loading csv dataframe of csv
heart_disease = pd.read_csv("/content/heart_disease_data.csv")
heart_disease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [26]:
# shape of data sset
heart_disease.shape

(303, 14)

In [27]:
# missing value
heart_disease.isna().sum()

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [28]:
# Duplicate value
heart_disease.duplicated().sum()

np.int64(1)

In [29]:
heart_disease['target'].value_counts()

,count
target,
1,165
0,138


1--> Defective Heart

0--> Healthy Heart

In [30]:
# spliting data set features and target
x = heart_disease.drop(columns='target', axis=1)
y = heart_disease['target']
print(x)


     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2  
2        2   0    

In [31]:
print(y)

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64


In [32]:
x = np.asarray(x)
y = np.asarray(y)

In [33]:
print(x)

[[63.  1.  3. ...  0.  0.  1.]
 [37.  1.  2. ...  0.  0.  2.]
 [41.  0.  1. ...  2.  0.  2.]
 ...
 [68.  1.  0. ...  1.  2.  3.]
 [57.  1.  0. ...  1.  1.  3.]
 [57.  0.  1. ...  1.  1.  2.]]


**Model Selection**

In [34]:
# list he models
models = [LogisticRegression(max_iter=10000), SVC(kernel='linear'), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [35]:
def compare_models_cross_validation():
  for model in models:
    cv_score = cross_val_score(model, x, y, cv=5)

    mean_accuracy = sum(cv_score)/len(cv_score)
    mean_accuracy = mean_accuracy*100
    mean_accuracy = round(mean_accuracy, 2)

    print('Cross Validation accuracies for the', model, '=', cv_score)
    print('Mean Accuracy score for the ', model, '=', mean_accuracy, '%')
    print('--------------------------------------------------------------------------------------------------------------')

In [36]:
compare_models_cross_validation()

Cross Validation accuracies for the LogisticRegression(max_iter=10000) = [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Mean Accuracy score for the  LogisticRegression(max_iter=10000) = 82.83 %
--------------------------------------------------------------------------------------------------------------
Cross Validation accuracies for the SVC(kernel='linear') = [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Mean Accuracy score for the  SVC(kernel='linear') = 82.83 %
--------------------------------------------------------------------------------------------------------------
Cross Validation accuracies for the KNeighborsClassifier() = [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Mean Accuracy score for the  KNeighborsClassifier() = 64.39 %
--------------------------------------------------------------------------------------------------------------
Cross Validation accuracies for the RandomForestClassifier(random_state=0) = [0.85245902 0.90163934 0.81967

In [37]:
# llist of model
models_list = [LogisticRegression(max_iter=10000), SVC(), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [38]:
# creating a dictionary that contains hyperparameter

model_hyperparameters = {
    'log_reg_hyperparameters': {
        'C' : [1,5,10,20]
    },

    'svc_hyperparameter': {
        'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
        'C' : [1,5,10,20]
    },

    'KNN_hyperparameters': {
        'n_neighbors' : [3,5,10]

    },

    'random_forest_hyperparameters' : {
        'n_estimators' : [10,20,50,100]

    }
}

In [39]:
type(model_hyperparameters)

dict

In [54]:
model_key=list(model_hyperparameters.keys())
print(model_key)

['log_reg_hyperparameters', 'svc_hyperparameter', 'KNN_hyperparameters', 'random_forest_hyperparameters']


In [52]:
model_hyperparameters['KNN_hyperparameters']


{'n_neighbors': [3, 5, 10]}

Applying GridSearchCV

In [59]:
def model_selection(list_of_models, hyperparameters_dictionary):
  result = []

  i = 0

  for model in list_of_models:
    key = model_key[i]

    params = hyperparameters_dictionary[key]
    i += 1

    print(model)
    print(params)

    classifier = GridSearchCV(model, params, cv=5)

    classifier.fit(x,y)

    result.append({
        'model used' : model,
        'highest score' : classifier.best_score_,
        'best hyperparameters' : classifier.best_params_
    })

  result_dataframe = pd.DataFrame(result, columns=['model used', 'highest score', 'best hyperparameters'])

  return result_dataframe

In [60]:
model_selection(models_list, model_hyperparameters)

LogisticRegression(max_iter=10000)
{'C': [1, 5, 10, 20]}
SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}


,model used,highest score,best hyperparameters
0,LogisticRegression(max_iter=10000),0.831585,{'C': 5}
1,SVC(),0.828306,"{'C': 1, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
3,RandomForestClassifier(random_state=0),0.838087,{'n_estimators': 100}
